# GEP-OnSSET GIS-Extraction Notebook for GEP-OnSSET

This is the GEP-OnSSET GIS extraction notebook. It replaces the [QGIS plugin](https://github.com/global-electrification-platform/Cluster-based_extraction_OnSSET/tree/master/Plugin) used in the online course.

The main purpose of this notebook is to facilitate the change of single datasets without running through the entire plugin. Using this notebook the user will be able to change however many datasets needed.

In order to run an OnSSET analysis the following datasets are needed:
* Admin boundaries
* Slope
* Global horizontal irradiation
* Travel time
* Wind velocity
* Clusters **(these clusters should include: The name of the study area, the amount of nighttime lights, population, population living in areas with nighttime light and an ID column)**. The clusters can be downloaded from [Energydata.info](https://energydata.info/) or generated directly using this [code](https://github.com/global-electrification-platform/Clustering_notebook).

In addition to this there are also some optional datasets that can be used in the analysis:
* Custom Demand - Can be generated using this [code](https://github.com/global-electrification-platform/CREDIT_layer). Description of the methodology is available [here](https://www.mdpi.com/1996-1073/12/7/1395) has been used.
* Substations
* Transformers
* ESPs (existing mini-grids)
* Adm0 & Adm1 boundary layers
* Mini/Small hydro
* Existing and planned HV-lines
* Existing and planned MV-lines 
* Road network

Below instructions for each cell follows. The cells marked with **(Mandatory)** in the title have to be run.

### Useful hints and common error messages
* Make sure that all input layers are using EPSG:4326 as the coordinate system
* Make sure that the "crs" in cell 2 is in a coordinate system using meters as the unit
* It is often useful to clip all the input layers to the country boundaries in order to reduce processing times
* Make sure that each dataset actually has some data within the country boundaries
* Some of the datasets require the user to choose values from a dropdown list below
* For hydro points and mini-grids, the vector layers need some specific column names to work
* In case a dataset still does not work, try opening it in QGIS and run the *Fix geometries* tool and save the new layer.
* If things do not work, it may be useful to go to the very top of this Jupyter Notebook and start again from cell 1


## Cell 1 - Importing necessary packages (Mandatory)

Packages to be used are imported from the funcs.ipynb.

In [1]:
%run funcs.ipynb
import time

## Cell 2 - Setting the target coordinate system (Mandatory)

When calculating distances it is important to choose a coordinate system that represents distances correctly in your area of interst. The coordinate system that is given below is the World Mercator, these coordinate system generally work well, but the distortions get larger as you move away from the equator.

In order to select your own coordinate system go to [epsg.io](http://epsg.io/) and type in your area of interest, this will give you a list of coordinate systems to choose from. Once you have selected your coordinate system replace the numbers below with the numbers from your coordinate system **(keep the "EPSG" part)**.

**NOTE** When selecting your coordinate system make sure that you select a system with the unit of meters, this is indicated for all systems on [epsg.io](http://epsg.io/)

In [2]:
crs = 'EPSG:3395'

## Cell 3 - Select the workspace and the administrative boundaries (Mandatory)

Define the workspace. The output layers will populate this folder. It is highly recommended to select an empty folder as your workspace.

For the administrative boundaries you will have to select an **Polygon** layer represeting your area of interest.
        

In [3]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

In [4]:
messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = gpd.read_file(filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*"))))

## Cell 4 - Select the population clusters (Mandatory)

Select the clusters to be used in the analysis

Please also indicate which column is representing the population data as this will be used later. 

In [5]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*"))))

# Keep only the clusters within the selected admin boundary
# clusters = gpd.clip(clusters, admin)

popunit = widgets.Dropdown(options=clusters.head(),
    value='Population',
    description='Population:',
    disabled=False)


In [6]:
display(popunit)
x = popunit.value

Dropdown(description='Population:', index=3, options=('Area', 'Country', 'cluster_id', 'Population', 'IsUrban'…

## Cell 5 - Select the Global Horizontal Irradiation (GHI) map - Raster map (Mandatory)

**If your settlement data already includes GHI data, skip to cell 8. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the ghi map that you wish to use in your analysis. This cell will extract the ghi values in your raster map to your clusters.

In [7]:
messagebox.showinfo('OnSSET', 'Select the Solar GHI map')
raster=filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

try:
    clusters = zonal_stat(raster, clusters, 'mean', 'GHI')
    print(time.ctime())
except rasterio.RasterioIOError as e:
    print('Could not process GHI, layer was not selected')
    print(e)

Sun Aug  4 19:12:09 2024


## Cell 6 - Select the Travel Time map - Raster map (Mandatory)
 
**If your settlement data already includes travel time data, skip to cell 9. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the travel time map that you wish to use in your analysis. This cell will extract the travel time values in your raster map to your clusters.

In [8]:
messagebox.showinfo('OnSSET', 'Select the Travel Time map')
raster=filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

try:
    clusters = zonal_stat(raster, clusters, 'mean', 'TravelTime')
    print(time.ctime())
except rasterio.RasterioIOError as e:
    print('Could not process Travel Time, layer was not selected')
    print(e)    

Sun Aug  4 19:12:32 2024


## Cell 7 - Select the Wind Velocity map - Raster map (Mandatory)

**If your settlement data already includes wind velocity data, skip to cell 10. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the wind velocity map that you wish to use in your analysis. This cell will extract the wind velocity values in your raster map to your clusters.

In [9]:
messagebox.showinfo('OnSSET', 'Select the Wind Speed map')
raster=filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

try:
    clusters = zonal_stat(raster, clusters, 'mean', 'WindVel')
    print(time.ctime())
except rasterio.RasterioIOError as e:
    print('Could not process Wind Speed, layer was not selected')
    print(e)  

Sun Aug  4 19:12:55 2024


## Cell 8 - Select the Night Lights map - Raster map (Mandatory)

**If your settlement data already includes night lights data, skip to cell 11. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the wind velocity map that you wish to use in your analysis. This cell will extract the wind velocity values in your raster map to your clusters.

In [10]:
messagebox.showinfo('OnSSET', 'Select the Night Lights map')
raster=filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

try:
    clusters = zonal_stat(raster, clusters, 'max', 'NightLight')
    print(time.ctime())
except rasterio.RasterioIOError as e:
    print('Could not process Night Lights, layer was not selected')
    print(e) 

Sun Aug  4 19:13:21 2024


## Cell 9 - Select the Custom Demand OR Demand index layer(s) - Raster map (Optional)

This cell may be used to extract the values from a) the Custom Residential Electricity Demand Indicative Target (CREDIT) layer **OR** b) the AHP classification process preceding that. Both options depend on raster layers that can be generated based on available data. 

You may refer to this [code](https://github.com/global-electrification-platform/CREDIT_layer) for additional information of how to generate these layers. 

**Note** that we also use two input rasters for urban/rural settlements indicatively. 

In [11]:
messagebox.showinfo('OnSSET', 'Select the Custom Demand map')
raster=filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

try:
    clusters = zonal_stat(raster, clusters, 'mean', 'CustomDemand')
    print(time.ctime())
except rasterio.RasterioIOError as e:
    print('Could not process Custom Demand, layer was not selected')
    print(e)    

Sun Aug  4 19:13:50 2024


## Cell 10 - Preparing to run the vector data (Mandatory)

**If you are planning on extracting any vector data (substations, transformers, hydro, MV-lines, HV-lines or roads) run this cell**. 

This cell reprojects the settlements to the coordinate system you specified above.

In [12]:
clusters = preparing_for_vectors(workspace, clusters, crs)

2024-08-04 19:13:52.143182


## Cell 11 - Substations - Vector point layer (Optional)

**If you do not have substations or wish to keep the ones already in your settlement file, skip to cell 14.**

Determines the distances between each settlement point to the closest substation. 

In [13]:
try:
    clusters = processing_points("Substation", admin, crs, workspace, clusters, mg_filter=False)
except fiona.errors.DriverError as e:
    print('Could not process Substations, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Substations. Check the coordinate system and that there are substations in the study area')
    print(e)

Could not process Substations, layer was not selected
: No such file or directory


## Cell 12 - Existing high voltage lines - Vector line layer (Optional)

**If you do not have existing high voltage lines or wish to keep the ones already in your settlement file, skip to cell 15.**

Determines the distances between each settlement point to the closest existing high voltage line. 

In [14]:
try:
    clusters = processing_lines("Existing_HV", admin, crs, workspace, clusters)
except fiona.errors.DriverError as e:
    print('Could not process Existing High Voltage lines, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Existing High Voltage lines. Check the coordinate system and that there are substations in the study area')
    print(e)

2024-08-04 19:14:23.311969


## Cell 13 - Planned high voltage lines - Vector line layer (Optional)

**If you do not have planned high voltage lines or wish to keep the ones already in your settlement file, skip to cell 16.**

Determines the distances between each settlement point to the closest planned high voltage line. 

In [15]:
try:
    clusters = processing_lines("Planned_HV", admin, crs, workspace, clusters)
except fiona.errors.DriverError as e:
    print('Could not process Planned High Voltage lines, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Planned High Voltage lines. Check the coordinate system and that there are substations in the study area')
    print(e)

2024-08-04 19:14:34.583660


## Cell 14 - Existing medium voltage lines - Vector line layer (Optional)

**If you do not have existing medium voltage lines or wish to keep the ones already in your settlement file, skip to cell 17.**

Determines the distances between each settlement point to the closest existing medium voltage line. 

In [16]:
try:
    clusters = processing_lines("Existing_MV", admin, crs, workspace, clusters)
except fiona.errors.DriverError as e:
    print('Could not process Existing Medium Voltage lines, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Existing Medium Voltage lines. Check the coordinate system and that there are substations in the study area')
    print(e)

2024-08-04 19:15:08.247253


## Cell 15 - Planned medium voltage lines - Vector line layer (Optional)

**If you do not have planned medium voltage lines or wish to keep the ones already in your settlement file, skip to cell 18.**

Determines the distances between each settlement point to the closest planned medium voltage line. 

In [17]:
try:
    clusters = processing_lines("Planned_MV", admin, crs, workspace, clusters)
except fiona.errors.DriverError as e:
    print('Could not process Planned Medium Voltage lines, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Planned Medium Voltage lines. Check the coordinate system and that there are substations in the study area')
    print(e)

Could not process Planned Medium Voltage lines, layer was not selected
: No such file or directory


## Cell 16 - Roads - Vector line layer (Optional)

**If you do not have roads or wish to keep the ones already in your settlement file, skip to cell 19.**

Determines the distances between each settlement point to the closest road. 

In [18]:
try:
    clusters = processing_lines("Road", admin, crs, workspace, clusters)
except fiona.errors.DriverError as e:
    print('Could not process Roads, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Roads. Check the coordinate system and that there are substations in the study area')
    print(e)

2024-08-04 19:18:06.048248


## Cell 17 - Service/distribution transformers - Vector point layer (Optional)

**If you do not have transformers or wish to keep the ones in the already in the settlement file, skip to cell 20** 

Determines the distances between each settlement point to the closest transformer. 

In [19]:
try:
    clusters = processing_points("Service Transformer", admin, crs, workspace, clusters, mg_filter=False)
except fiona.errors.DriverError as e:
    print('Could not process Service transformers, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Service transformers. Check the coordinate system and that there are substations in the study area')
    print(e)

2024-08-04 19:18:26.318615


## Cell 18 and 19 - Hydro points - Vector point layer (Optional)

**If you do not have new hydro power points skip to next step

**In Cell 20** Select the hydro point layer you wish to use. It is important to have a column representing the power output for each hydro point in your dataset. After selecting the column you will also have to select the unit (W, kW or MW). 

**In Cell 21** When everything is selected in cell 22, run cell 23 in order to determine the distance to the closest hydro point for each settlement.

**Note** that the layer should be vector-points and not vector-multipoints. In case of the latter please do convert that in Qgis (or similar) using SAGA function "Convert multipoints to points".

In [20]:
try:
    messagebox.showinfo('OnSSET', 'Select the Hydropower map')
    hydro=gpd.read_file(filedialog.askopenfilename(title = "Select Hydro map", filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*"))))

    hydropower = widgets.Dropdown(options=hydro.head(),
        value='PowerMW',
        description='Hydropower:',
        disabled=False)

    display(hydropower)

    hydrounit = widgets.Dropdown(options=['W', 'kW', 'MW'],
        value='MW',
        description='Unit:',
        disabled=False)

    display(hydrounit)
except fiona.errors.DriverError as e:
    print('Could not process Hydro points, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Hydro points. Check the coordinate system and that there are substations in the study area')
    print(e)

Dropdown(description='Hydropower:', index=7, options=('OBJECTID', 'river_ID', 'elev', 'head', 'discharge_', 'p…

Dropdown(description='Unit:', index=2, options=('W', 'kW', 'MW'), value='MW')

In [21]:
try:
    clusters = processing_hydro(admin, crs, workspace, clusters, hydro, hydropower.value, hydrounit.value)
except fiona.errors.DriverError as e:
    print('Could not process Hydro points, layer was not selected')
    print(e)
except NameError as e:
    print('Could not process Hydro points, there was an error or layer was not selected')
    print(e)

2024-08-04 19:18:38.737460


# Extra datasets that can be used to improve the analysis

## Cell 20 - Existing mini-grids - Vector point layer (Optional extra)

This function extracts the nearest mini-grid to each clusters and assigns key characteristics (e.g. name, MV network status, type).

In [22]:
try:
    clusters = processing_points("MG", admin, crs, workspace, clusters, mg_filter=True)
except fiona.errors.DriverError as e:
    print('Could not process Mini-Grids, layer was not selected')
    print(e)
except ValueError as e:
    print('Could not process Mini-Grids. Check the coordinate system and that there are mini-grids in the study area')
    print(e)

2024-08-04 19:18:50.726424


## Cell 21 - Extracting admin 1 name to clusters - Vector polygon layer (Optional extra)

This function extracts the admin level 1 name to each cluster based on spatial overlay. 

**Please do provide the right column name (e.g. "adm1_name") in the first variable**

In [23]:
admin_1_name = "adm1_name"

try:
    clusters = get_admin1_name(clusters, admin_1_name, crs)
except fiona.errors.DriverError as e:
    print('Could not process Admin 1, layer was not selected')
    print(e)

2024-08-04 19:19:25.880660


## Cell 22 - Conditioning & Export (Mandatory)

This is the final cell in the extraction. This cell has to be run.

In [24]:
clusters = conditioning(clusters, workspace, popunit.value)

2024-08-04 19:19:26.659028
The extraction file is now ready for review & use in the workspace directory as 'GEP-OnSSET_InputFile.csv'!


In [25]:
#clusters.to_csv("GEP_OnSSET_InputFile.csv", index=False)